### Notebook setup
Run the following cells to initiate notebook

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import numpy as np
import io
import requests
import random
import shutil
from utils import read_pred_file
import time

In [ ]:
API_URL = "http://api:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)

# Download some relevant images and predictions
if not os.path.isdir("selection-true-positives"):
    print("Images not found, dowloading ...")
    url = "https://github.com/pyronear/pyro-envdev/releases/download/v0.0.1/selection-true-positives.zip"
    output_path = "selection-true-positives.zip"

    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raises an error for bad status codes

    with open(output_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    zip_path = "selection-true-positives.zip"
    extract_dir = "selection-true-positives"  # Current directory

    shutil.unpack_archive(zip_path, extract_dir, 'zip')

    print("Extraction completed.")


# Send detection continuoulsy
For a given CAMERA_ID, the following cell will continuously send 1 detection periodically (each IMG_FREQ_SECONDS) 

To stop the cell from running you can click on the "stop" button in the ribbon above the notebook.


In [ ]:
CAMERA_ID = 1
SEQUENCE_CENTER_AZIMUTH = 180
IMG_FREQ_SECONDS = 30


camera_token = get_camera_token(API_URL, CAMERA_ID, admin_access_token)
camera_client  = Client(camera_token, API_URL)

seq_folder = "selection-true-positives/2025-06-13T14-50-34_mostazal--2-180_sequence-1195"

imgs = glob.glob(f"{seq_folder}/images/*")
imgs.sort()
preds = glob.glob(f"{seq_folder}/labels_predictions/*")
preds.sort()

while True:
    for img_file, pred_file in zip(imgs, preds):
        stream = io.BytesIO()
        im = Image.open(img_file)
        im.save(stream, format="JPEG", quality=80)

        bboxes = read_pred_file(pred_file)

        response = camera_client.create_detection(stream.getvalue(), SEQUENCE_CENTER_AZIMUTH, bboxes)
        # Force a KeyError if the request failed
        response.json()["id"]
        print("detection sent")
        time.sleep(IMG_FREQ_SECONDS)
    